# VAE（Variational Autoencoder）结构细节
VAE 由编码器（Encoder）、解码器（Decoder） 以及量化相关层（quant_conv、post_quant_conv）组成，核心功能是将图像映射到潜在空间（编码）并从潜在空间重建图像（解码）。
1. 编码器（Encoder）
输入处理：接收 3 通道图像，首先通过conv_in（Conv2d(3, 128, kernel_size=3, stride=1, padding=1)）将通道数转为 128。
下采样模块（down_blocks）：包含 4 个DownEncoderBlock2D，逐步降低空间维度并提升通道数：
第 0 块：2 个ResnetBlock2D（通道数保持 128），配合Downsample2D（Conv2d(128, 128, kernel_size=3, stride=2)）实现下采样（尺寸减半）。
第 1 块：第 1 个ResnetBlock2D将通道从 128 升至 256（通过conv_shortcut），第 2 个保持 256；配合Downsample2D（Conv2d(256, 256, kernel_size=3, stride=2)）下采样。
第 2 块：第 1 个ResnetBlock2D将通道从 256 升至 512，第 2 个保持 512；配合Downsample2D（Conv2d(512, 512, kernel_size=3, stride=2)）下采样。
第 3 块：2 个ResnetBlock2D保持通道数 512，无下采样。
中间块（mid_block）：UNetMidBlock2D，包含 1 个Attention（自注意力，基于 512 维特征）和 2 个ResnetBlock2D（保持 512 通道），增强特征交互。
输出处理：通过conv_norm_out（GroupNorm(32, 512)）、conv_act（SiLU 激活），最后经conv_out（Conv2d(512, 8, kernel_size=3, padding=1)）输出 8 通道特征（含均值和方差，用于潜在空间量化）。
2. 解码器（Decoder）
输入处理：接收 4 通道潜在向量（量化后），通过conv_in（Conv2d(4, 512, kernel_size=3, padding=1)）将通道转为 512。
上采样模块（up_blocks）：包含 4 个UpDecoderBlock2D，逐步提升空间维度并降低通道数：
第 0-1 块：各含 3 个ResnetBlock2D（保持 512 通道），配合Upsample2D（插值 +Conv2d(512, 512)）实现上采样（尺寸加倍）。
第 2 块：第 1 个ResnetBlock2D将通道从 512 降至 256（通过conv_shortcut），后 2 个保持 256；配合Upsample2D（Conv2d(256, 256)）上采样。
第 3 块：第 1 个ResnetBlock2D将通道从 256 降至 128（通过conv_shortcut），后 2 个保持 128；无上采样。
中间块（mid_block）：结构同编码器的UNetMidBlock2D，含 1 个Attention和 2 个ResnetBlock2D（512 通道）。
输出处理：通过conv_norm_out（GroupNorm(32, 128)）、conv_act（SiLU 激活），最后经conv_out（Conv2d(128, 3, kernel_size=3, padding=1)）输出 3 通道重建图像。
3. 量化相关层
quant_conv：Conv2d(8, 8, kernel_size=1)，处理编码器输出的 8 通道特征（均值和方差）。
post_quant_conv：Conv2d(4, 4, kernel_size=1)，处理量化后的 4 通道潜在向量，作为解码器输入。

# U-Net（UNet2DConditionModel）结构细节
U-Net 是扩散模型的核心，负责从噪声中逐步生成图像，结合时间步信息和文本条件（通过交叉注意力）。结构包括下采样模块（down_blocks）、上采样模块（up_blocks）、中间块（mid_block） 及时间嵌入处理。
1. 输入与时间嵌入
输入处理：接收 4 通道潜在向量（与 VAE 潜在空间对应），通过conv_in（Conv2d(4, 320, kernel_size=3, padding=1)）转为 320 通道。
时间嵌入：time_proj生成时间步编码，经time_embedding（含 2 层线性层 + SiLU，Linear(320→1280→1280)）输出 1280 维时间特征，用于后续残差块调制。
2. 下采样模块（down_blocks）
含 4 个块，逐步降低空间维度、提升通道数，并引入文本条件交互：
第 0-2 块（CrossAttnDownBlock2D）：
注意力机制：2 个Transformer2DModel，每个含BasicTransformerBlock（自注意力 + 交叉注意力）。交叉注意力的key/value来自 1024 维文本特征（如 CLIP 文本编码），实现文本 - 图像特征对齐。
残差块：ResnetBlock2D，结合时间嵌入（通过time_emb_proj将 1280 维时间特征映射到当前通道）。第 0 块保持 320 通道，第 1 块从 320 升至 640，第 2 块从 640 升至 1280。
下采样：Downsample2D（Conv2d(通道, 通道, kernel_size=3, stride=2, padding=1)），尺寸减半。
第 3 块（DownBlock2D）：仅含 2 个ResnetBlock2D（保持 1280 通道），无注意力机制，继续下采样。
3. 上采样模块（up_blocks）
含 4 个块，逐步提升空间维度、降低通道数，融合下采样阶段的跳跃连接特征：
第 0 块（UpBlock2D）：3 个ResnetBlock2D，处理来自下采样的 2560 通道特征（跳跃连接 + 当前特征），将通道降至 1280；配合Upsample2D（插值 + 卷积）上采样（尺寸加倍）。
第 1-3 块（CrossAttnUpBlock2D）：
注意力机制：3 个Transformer2DModel，同下采样的交叉注意力结构，持续结合文本特征。
残差块：ResnetBlock2D融合跳跃连接与当前特征，通道逐步从 1280 降至 640（第 1 块）、再降至 320（第 2-3 块）。
上采样：Upsample2D（卷积 + 插值），尺寸加倍（第 1-2 块）。
4. 中间块（mid_block）
UNetMidBlock2DCrossAttn，位于下采样与上采样之间，增强全局特征交互：
含 1 个Transformer2DModel（自注意力 + 交叉注意力，文本特征驱动）和 2 个ResnetBlock2D（1280 通道，结合时间嵌入）。
5. 输出处理
通过conv_norm_out（GroupNorm(32, 320)）、conv_act（SiLU 激活），最后经conv_out（Conv2d(320, 4, kernel_size=3, padding=1)）输出 4 通道特征（用于更新噪声）。
核心共性与差异
共性：均采用 U 型结构（下采样 + 上采样），依赖ResnetBlock2D提取特征，使用GroupNorm归一化和 SiLU 激活。
差异：VAE 聚焦图像 - 潜在空间映射，无注意力机制；U-Net 引入时间嵌入和文本交叉注意力，用于扩散过程的噪声预测。

# 核心共性与差异
共性：均采用 U 型结构（下采样 + 上采样），依赖ResnetBlock2D提取特征，使用GroupNorm归一化和 SiLU 激活。
差异：VAE 聚焦图像 - 潜在空间映射，无注意力机制；U-Net 引入时间嵌入和文本交叉注意力，用于扩散过程的噪声预测。